The `logging` module is a powerful and flexible framework for tracking events in your software. It allows you to record messages of various severity levels, output these messages to different destinations, and format them in a customizable way.

### A simple example

In [1]:
import logging

logging.info('This is an info message')
logging.warning('This is a warning message')

In this very basic example, we used `info()` and `warning()` functions in the `logging` module. These functions are named after the level or severity of the events they are used to track. The standard levels and their applicability are described below (in increasing order of severity):

   - `DEBUG`: Detailed information, typically of interest only when diagnosing problems.
   - `INFO`: Confirmation that things are working as expected.
   - `WARNING`: An indication that something unexpected happened, or indicative of some problem in the near future (e.g., ‘disk space low’). The software is still working as expected.
   - `ERROR`: Due to a more serious problem, the software has not been able to perform some function.
   - `CRITICAL`: A very serious error, indicating that the program itself may be unable to continue running.

The default level is `WARNING`, which means that only events of this level and above will be tracked, unless the logging package is configured to do otherwise. This is why only the output of the `warning()` function is visible above.

The example above is good enough for a simple application. In larger programs you’ll usually want to control the logging configuration explicitly however - so for that reason as well as others, it’s better to create loggers and call their methods.

### Logging to a file

A very common situation is that of recording logging events in a file:

```python
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(filename='example.log', encoding='utf-8', level=logging.DEBUG)
logger.debug('This message should go to the log file')
logger.info('So should this')
logger.warning('And this, too')
logger.error('And non-ASCII stuff, too, like Øresund and Malmö')
```

This will produce the following output, in the `example.log` file:
```
DEBUG:__main__:This message should go to the log file
INFO:__main__:So should this
WARNING:__main__:And this, too
ERROR:__main__:And non-ASCII stuff, too, like Øresund and Malmö
```

The example, broken down in steps:

1. **Importing the logging module:**
   ```python
   import logging
   ```
   
   First, the logging module is imported.

2. **Creating a logger:**
   ```python
   logger = logging.getLogger(__name__)
   ```
   
   This line creates a logger object named after the current module's name (`__name__`). The `__name__` variable is a special built-in variable that evaluates to the name of the current module. Using `__name__` ensures that each module can have its own logger if this code is part of a larger application.

3. **Configuring the logging system:**
   ```python
   logging.basicConfig(filename='example.log', encoding='utf-8', level=logging.DEBUG)
   ```
   
   This line configures the logging system to write log messages to a file named `example.log`. The `encoding='utf-8'` ensures that the log file will be encoded in UTF-8, which is important for properly handling non-ASCII characters. The `level=logging.DEBUG` sets the threshold for the logger to the DEBUG level, meaning that all messages at this level and above (INFO, WARNING, ERROR, and CRITICAL) will be logged.

4. **Logging messages:**
   ```python
   logger.debug('This message should go to the log file')
   logger.info('So should this')
   logger.warning('And this, too')
   logger.error('And non-ASCII stuff, too, like Øresund and Malmö')
   ```
   
   These lines send log messages at various severity levels to the logger.

If you run the above script several times, the messages from successive runs are appended to the file `example.log`. If you want each run to start afresh, not remembering the messages from earlier runs, you can specify the filemode argument, by changing the call in the above example to:

```python
logging.basicConfig(filename='example.log', filemode='w', level=logging.DEBUG)
```

### Logging variable data

To log variable data, use a format string for the event description message and append the variable data as arguments.

In [2]:
name = "Jane"
age = 20
logging.warning("%s is %d years old", name, age)

Never use string formatting instead of sending the variables as parameters as the example above. String formatting will always build a string, whereas when sending variables as values to the log function, the string will be created only if that level is actually logged.

### Logging exceptions


In [3]:
try:
    10 / 0
except ZeroDivisionError as ex:
    logging.error("Error occurred", exc_info=True)

ERROR:root:Error occurred
Traceback (most recent call last):
  File "/var/folders/s2/_752pzb50tg00mv9ggtr6p8c0000gn/T/ipykernel_52524/951162099.py", line 2, in <module>
    10 / 0
    ~~~^~~
ZeroDivisionError: division by zero


### Changing the format of displayed messages
To change the format which is used to display messages, you need to specify the format you want to use:

```python
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)
logging.debug('This message should appear on the console')
logging.info('So should this')
logging.warning('And this, too')
```
This would print:
```
DEBUG:This message should appear on the console
INFO:So should this
WARNING:And this, too
```
For a full set of things that can appear in format strings, take a look [here](https://docs.python.org/3/library/logging.html#logrecord-attributes).

### Displaying the date/time in messages

To display the date and time of an event, you would place ‘%(asctime)s’ in your format string:

```python
import logging
logging.basicConfig(format='%(asctime)s %(message)s')
logging.warning('is when this event was logged.')
```
which should print something like this:
```
2024-06-05 11:50:36,634 is when this event was logged.
```
This uses the default date time string representation. If you need more control over the formatting of the date/time, provide a datefmt argument to basicConfig, as in this example:
```python
import logging
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
logging.warning('is when this event was logged.')
```
which would display something like this:
```
05 Jun 2024 11:53:27 AM is when this event was logged.
```

### Exercises 1
1. Add logging to [process_files.py](./process_files.py). Change all `print` messages to logs. Include current datetime (choose a format different from the one used in the examples above), log level and message. Add other logs; try to use multiple log levels.

### Advanced Logging in Python: Custom Loggers, Handlers, and Formatters

In the previous section, we covered basic logging using a single logger and the `basicConfig` method. Now, let's dive into more advanced logging techniques by creating custom loggers, handlers, and formatters. This approach provides greater flexibility and control over your logging system, allowing you to direct log messages to different destinations and format them according to your needs.

#### Step 1: Import the `logging` Module

First, ensure you have the `logging` module imported, as it provides all the necessary functionalities for logging.

```python
import logging
```

#### Step 2: Create a Custom Logger

A logger is the main entry point of the logging system. To create a custom logger, use the `logging.getLogger(name)` method. The `name` parameter typically represents the module's name.

```python
logger = logging.getLogger('custom_logger')
logger.setLevel(logging.DEBUG)
```

Here, the logger is named `'custom_logger'`, and its level is set to `DEBUG`. This means it will capture all log messages of level `DEBUG` and above.

#### Step 3: Create Handlers

Handlers are responsible for sending log messages to their final destination. Common handlers include:

- **StreamHandler**: Sends log messages to the console.
- **FileHandler**: Sends log messages to a file.

```python
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)

file_handler = logging.FileHandler('app.log', encoding='utf-8')
file_handler.setLevel(logging.INFO)
```

In this example, we create a `StreamHandler` to log messages to the console and a `FileHandler` to log messages to a file named `app.log`. The file handler uses UTF-8 encoding to handle non-ASCII characters properly.

#### Step 4: Create Formatters

Formatters define the layout of the log messages. You can customize the format to include details like the timestamp, logger name, log level, and the message itself.

```python
console_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_formatter = logging.Formatter('%(asctime)s:%(name)s:%(levelname)s - %(message)s')

console_handler.setFormatter(console_formatter)
file_handler.setFormatter(file_formatter)
```

Here, we create a formatter for both the console and file handlers. The format string `'%(asctime)s - %(name)s - %(levelname)s - %(message)s'` includes the timestamp, logger name, log level, and message.

#### Step 5: Add Handlers to the Logger

Attach the handlers to the logger to enable it to send log messages to the specified destinations.

```python
logger.addHandler(console_handler)
logger.addHandler(file_handler)
```

#### Step 6: Log Messages

You can now use the logger to log messages at various levels.

```python
logger.debug('Debug message')
logger.info('Info message')
logger.warning('Warning message')
logger.error('Error message')
logger.critical('Critical message')
```

#### Full Example

Here is the complete example combining all the steps:

```python
import logging

# Step 2: Create a custom logger
logger = logging.getLogger('custom_logger')
logger.setLevel(logging.DEBUG)

# Step 3: Create handlers
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)

file_handler = logging.FileHandler('app.log', encoding='utf-8')
file_handler.setLevel(logging.INFO)

# Step 4: Create formatters and add them to the handlers
console_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_formatter = logging.Formatter('%(asctime)s:%(name)s:%(levelname)s - %(message)s')

console_handler.setFormatter(console_formatter)
file_handler.setFormatter(file_formatter)

# Step 5: Add the handlers to the logger
logger.addHandler(console_handler)
logger.addHandler(file_handler)

# Step 6: Log messages
logger.debug('Debug message')
logger.info('Info message')
logger.warning('Warning message')
logger.error('Error message')
logger.critical('Critical message')
```

#### Customizing Further

You can further customize your logging setup by creating different handlers for different log levels or destinations. For example, you might want to log debug messages to the console but only log errors to a file.

##### Example: Different Handlers for Different Levels

```python
import logging

# Create a custom logger
logger = logging.getLogger('advanced_logger')
logger.setLevel(logging.DEBUG)

# Create handlers
console_handler = logging.StreamHandler()
file_handler = logging.FileHandler('errors.log', encoding='utf-8')

# Set levels for handlers
console_handler.setLevel(logging.DEBUG)
file_handler.setLevel(logging.ERROR)

# Create formatters and add them to handlers
console_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_formatter = logging.Formatter('%(asctime)s:%(name)s:%(levelname)s - %(message)s')

console_handler.setFormatter(console_formatter)
file_handler.setFormatter(file_formatter)

# Add handlers to the logger
logger.addHandler(console_handler)
logger.addHandler(file_handler)

# Log messages
logger.debug('This will go to the console')
logger.info('This will also go to the console')
logger.error('This will go to both the console and the file')
```

In this example, only messages at the ERROR level and above will be written to `errors.log`, while all messages (DEBUG level and above) will be printed to the console.

### Exercises 2

1. Add advanced logging to [process_files.py](./process_files.py). Use multiple handlers with different logging levels and different formatters.